##生成数据集##

In [12]:
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
# features训练数据,labels标签

##读取数据##

In [21]:
import torch.utils.data as Data
# 由于data常用作变量名，我们将导入的data模块用Data代替

batch_size = 10
dataset = Data.TensorDataset(features, labels)
# 将训练数据和标签进行组合
data_itr = Data.DataLoader(dataset, batch_size, shuffle=True)
print(data_itr)
# 随机读取小批量
# shuffle:重新洗牌

##定义模型##

通过torch.nn模块定义模型,nn模块的核心数据结构是nn.Module,nn.Module可以表示一个层或者网络

In [14]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)   # 输入特征,输出特征
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
# num_inputs 在生成数据集时已经定义好了
print(net)


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


##初始化模型参数##

In [17]:
from torch.nn import init
init.normal_(net.linear.weight, mean=0, std=0.01)
# 使用正态分布的随机采样对模型权重进行随机初始化
init.constant_(net.linear.bias, val=0)
# bias需要单独设置


Parameter containing:
tensor([0.], requires_grad=True)

##定义损失函数##
使用torch.nn中提供的均方误差损失作为损失函数

In [18]:
loss = nn.MSELoss()





##定义优化算法##

In [19]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(net.parameters())
print(optimizer)




<generator object Module.parameters at 0x7fd0fc8a9ed0>
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


**调整学习率**

In [20]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1




##训练模型##

In [26]:
num_epoch = 3   # 便利数据集的次数
for epoch in range(1, num_epoch+1):
    for X, y in data_itr:   # 按照batchsize将所有数据取出
        output = net.linear(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()   # 先将以前的梯度清零
        l.backward()            # 然后计算梯度并反向传播
        optimizer.step() 
    print('epoch %d, loss: %f' % (epoch, l.item()))# 用了optimizer.step()之后模型才会更新


epoch 1, loss: 0.386047
epoch 2, loss: 0.091308
epoch 3, loss: 0.031657
